In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

In [ ]:
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False, num_workers=2)

# Class labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Helper function for inline image display
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = dataiter.next()

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)
print('  '.join(classes[labels[j]] for j in range(4)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# PyTorch models inherit from torch.nn.Module
class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = GarmentClassifier()

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

# NB: Loss functions expect data in batches, so we're creating batches of 4
# Represents the model's confidence in each of the 10 classes for a given input
dummy_outputs = torch.rand(4, 10)
# Represents the correct class among the 10 being tested
dummy_labels = torch.tensor([1, 5, 3, 7])

print(dummy_outputs)
print(dummy_labels)

loss = loss_fn(dummy_outputs, dummy_labels)
print('Total loss for this batch: {}'.format(loss.item()))

In [ ]:
# Optimizers specified in the torch.optim package
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [ ]:
saved_model = GarmentClassifier()
saved_model.load_state_dict(torch.load(PATH))

In [28]:
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as dset
from torchvision import datasets, transforms

In [34]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('GPU State:', device)

GPU State: cuda:0


In [35]:
# Transform
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,)),]
)

In [39]:
# Data
trainSet = datasets.MNIST(root='MNIST', download=True, train=True, transform=transform)
testSet = datasets.MNIST(root='MNIST', download=True, train=False, transform=transform)
trainLoader = dset.DataLoader(trainSet, batch_size=64, shuffle=True)
testLoader = dset.DataLoader(testSet, batch_size=64, shuffle=False)

In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [37]:
trainLoader = dset.DataLoader(x_train, batch_size=64, shuffle=True)
testLoader = dset.DataLoader(x_test, batch_size=64, shuffle=False)

In [11]:
featuresTrain = torch.from_numpy(features_train)
targetsTrain = torch.from_numpy(targets_train).type(torch.LongTensor) # data type is long

featuresTest = torch.from_numpy(features_test)
targetsTest = torch.from_numpy(targets_test).type(torch.LongTensor) # data type is long

AttributeError: 'DataLoader' object has no attribute 'size'

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = batch_size, shuffle = True)

In [5]:
# Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_features=784, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=10),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, input):
        return self.main(input)


net = Net().to(device)
print(net)

Net(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): LogSoftmax(dim=1)
  )
)


In [6]:
# Parameters
epochs = 3
lr = 0.002
criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.002, momentum=0.9)

In [40]:
# Train
for epoch in range(epochs):
    running_loss = 0.0

    for times, data in enumerate(trainLoader):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.view(inputs.shape[0], -1)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Foward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if times % 100 == 99 or times+1 == len(trainLoader):
            print('[%d/%d, %d/%d] loss: %.3f' % (epoch+1, epochs, times+1, len(trainLoader), running_loss/2000))

print('Training Finished.')

[1/3, 100/938] loss: 0.017
[1/3, 200/938] loss: 0.028
[1/3, 300/938] loss: 0.038
[1/3, 400/938] loss: 0.048
[1/3, 500/938] loss: 0.058
[1/3, 600/938] loss: 0.067
[1/3, 700/938] loss: 0.076
[1/3, 800/938] loss: 0.085
[1/3, 900/938] loss: 0.095
[1/3, 938/938] loss: 0.098
[2/3, 100/938] loss: 0.008
[2/3, 200/938] loss: 0.016
[2/3, 300/938] loss: 0.024
[2/3, 400/938] loss: 0.032
[2/3, 500/938] loss: 0.040
[2/3, 600/938] loss: 0.047
[2/3, 700/938] loss: 0.056
[2/3, 800/938] loss: 0.063
[2/3, 900/938] loss: 0.071
[2/3, 938/938] loss: 0.074
[3/3, 100/938] loss: 0.007
[3/3, 200/938] loss: 0.014
[3/3, 300/938] loss: 0.021
[3/3, 400/938] loss: 0.028
[3/3, 500/938] loss: 0.035
[3/3, 600/938] loss: 0.042
[3/3, 700/938] loss: 0.049
[3/3, 800/938] loss: 0.055
[3/3, 900/938] loss: 0.061
[3/3, 938/938] loss: 0.064
Training Finished.


In [41]:
# Test
correct = 0
total = 0

with torch.no_grad():
    for data in testLoader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(inputs.shape[0], -1)

        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100*correct / total))

class_correct = [0 for i in range(10)]
class_total = [0 for i in range(10)]

with torch.no_grad():
    for data in testLoader:
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.view(inputs.shape[0], -1)

        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
            print(class_correct)
            print(class_total)

for i in range(10):
    print('Accuracy of %d: %3f' % (i, (class_correct[i]/class_total[i])))

Accuracy of the network on the 10000 test images: 95 %
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 1, 0, 0, 0, 0, 1, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 1, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 1, 0, 0]
[1, 1, 1, 0, 1, 0, 0, 1, 0, 0]
[1, 1, 1, 0, 1, 0, 0, 1, 0, 0]
[1, 2, 1, 0, 1, 0, 0, 1, 0, 0]
[1, 2, 1, 0, 1, 0, 0, 1, 0, 0]
[1, 2, 1, 0, 2, 0, 0, 1, 0, 0]
[1, 2, 1, 0, 2, 0, 0, 1, 0, 0]
[1, 2, 1, 0, 2, 0, 0, 1, 0, 1]
[1, 2, 1, 0, 2, 0, 0, 1, 0, 1]
[1, 2, 1, 0, 2, 0, 0, 1, 0, 1]
[1, 2, 1, 0, 2, 1, 0, 1, 0, 1]
[1, 2, 1, 0, 2, 0, 0, 1, 0, 2]
[1, 2, 1, 0, 2, 1, 0, 1, 0, 2]
[1, 2, 1, 0, 2, 0, 0, 2, 0, 2]
[1, 2, 1, 0, 2, 1, 0, 2, 0, 2]
[1, 2, 1, 0, 3, 0, 0, 2, 0, 2]
[1, 2, 1, 0, 3, 1, 0, 2, 0, 2]
[1, 2, 1, 0, 3, 0, 1, 2, 0, 2]
[1, 2, 1, 0, 3, 1, 1, 2, 0, 2]
[1, 2, 1, 0, 4, 0, 1, 2, 0, 2]
[1, 2, 1, 0, 4, 1, 1, 2, 0, 2]
[1, 2, 1, 1, 4, 0, 1, 2, 0, 2]
[1, 2, 1, 1, 4, 1, 1, 2, 0, 2]
[2, 2, 1, 1, 4,

[13, 16, 17, 11, 11, 7, 12, 12, 8, 10]
[13, 16, 18, 13, 12, 8, 13, 12, 10, 13]
[13, 16, 17, 11, 12, 7, 12, 12, 8, 10]
[13, 16, 18, 13, 13, 8, 13, 12, 10, 13]
[13, 17, 17, 11, 12, 7, 12, 12, 8, 10]
[13, 17, 18, 13, 13, 8, 13, 12, 10, 13]
[13, 17, 17, 11, 12, 7, 12, 13, 8, 10]
[13, 17, 18, 13, 13, 8, 13, 13, 10, 13]
[13, 17, 17, 11, 12, 7, 12, 13, 8, 11]
[13, 17, 18, 13, 13, 8, 13, 13, 10, 14]
[13, 17, 17, 11, 12, 7, 13, 13, 8, 11]
[13, 17, 18, 13, 13, 8, 14, 13, 10, 14]
[13, 18, 17, 11, 12, 7, 13, 13, 8, 11]
[13, 18, 18, 13, 13, 8, 14, 13, 10, 14]
[13, 19, 17, 11, 12, 7, 13, 13, 8, 11]
[13, 19, 18, 13, 13, 8, 14, 13, 10, 14]
[13, 19, 18, 11, 12, 7, 13, 13, 8, 11]
[13, 19, 19, 13, 13, 8, 14, 13, 10, 14]
[13, 19, 18, 11, 13, 7, 13, 13, 8, 11]
[13, 19, 19, 13, 14, 8, 14, 13, 10, 14]
[13, 19, 18, 11, 13, 7, 13, 13, 9, 11]
[13, 19, 19, 13, 14, 8, 14, 13, 11, 14]
[13, 20, 18, 11, 13, 7, 13, 13, 9, 11]
[13, 20, 19, 13, 14, 8, 14, 13, 11, 14]
[13, 20, 18, 11, 13, 7, 13, 14, 9, 11]
[13, 20, 19, 

[24, 31, 29, 18, 24, 14, 20, 23, 22, 19]
[24, 31, 30, 20, 25, 17, 22, 24, 24, 24]
[24, 31, 29, 18, 24, 14, 21, 23, 22, 19]
[24, 31, 30, 20, 25, 17, 23, 24, 24, 24]
[24, 31, 30, 18, 24, 14, 21, 23, 22, 19]
[24, 31, 31, 20, 25, 17, 23, 24, 24, 24]
[24, 31, 30, 19, 24, 14, 21, 23, 22, 19]
[24, 31, 31, 21, 25, 17, 23, 24, 24, 24]
[24, 31, 30, 19, 24, 14, 21, 24, 22, 19]
[24, 31, 31, 21, 25, 17, 23, 25, 24, 24]
[24, 31, 30, 19, 24, 14, 21, 24, 23, 19]
[24, 31, 31, 21, 25, 17, 23, 25, 25, 24]
[24, 31, 30, 19, 25, 14, 21, 24, 23, 19]
[24, 31, 31, 21, 26, 17, 23, 25, 25, 24]
[24, 31, 30, 19, 25, 14, 21, 25, 23, 19]
[24, 31, 31, 21, 26, 17, 23, 26, 25, 24]
[24, 31, 30, 19, 25, 14, 21, 26, 23, 19]
[24, 31, 31, 21, 26, 17, 23, 27, 25, 24]
[24, 31, 30, 19, 25, 14, 21, 26, 23, 20]
[24, 31, 31, 21, 26, 17, 23, 27, 25, 25]
[24, 31, 30, 20, 25, 14, 21, 26, 23, 20]
[24, 31, 31, 22, 26, 17, 23, 27, 25, 25]
[24, 31, 30, 21, 25, 14, 21, 26, 23, 20]
[24, 31, 31, 23, 26, 17, 23, 27, 25, 25]
[24, 31, 30, 21,

[29, 44, 41, 32, 39, 22, 29, 36, 34, 29]
[30, 45, 44, 34, 41, 26, 33, 37, 36, 35]
[29, 44, 41, 33, 39, 22, 29, 36, 34, 29]
[30, 45, 44, 35, 41, 26, 33, 37, 36, 35]
[29, 44, 41, 33, 40, 22, 29, 36, 34, 29]
[30, 45, 44, 35, 42, 26, 33, 37, 36, 35]
[29, 44, 41, 33, 41, 22, 29, 36, 34, 29]
[30, 45, 44, 35, 43, 26, 33, 37, 36, 35]
[29, 44, 41, 34, 41, 22, 29, 36, 34, 29]
[30, 45, 44, 36, 43, 26, 33, 37, 36, 35]
[29, 44, 41, 34, 41, 22, 29, 36, 35, 29]
[30, 45, 44, 36, 43, 26, 33, 37, 37, 35]
[29, 44, 41, 34, 41, 22, 29, 36, 35, 30]
[30, 45, 44, 36, 43, 26, 33, 37, 37, 36]
[29, 44, 42, 34, 41, 22, 29, 36, 35, 30]
[30, 45, 45, 36, 43, 26, 33, 37, 37, 36]
[29, 44, 42, 35, 41, 22, 29, 36, 35, 30]
[30, 45, 45, 37, 43, 26, 33, 37, 37, 36]
[29, 44, 42, 35, 41, 22, 29, 36, 35, 31]
[30, 45, 45, 37, 43, 26, 33, 37, 37, 37]
[29, 44, 42, 35, 42, 22, 29, 36, 35, 31]
[30, 45, 45, 37, 44, 26, 33, 37, 37, 37]
[29, 44, 42, 35, 42, 22, 29, 36, 35, 31]
[30, 45, 45, 37, 44, 27, 33, 37, 37, 37]
[30, 44, 42, 35,

[42, 59, 51, 50, 55, 27, 33, 51, 40, 40]
[43, 60, 55, 54, 58, 32, 37, 52, 43, 47]
[42, 59, 51, 50, 55, 27, 33, 51, 40, 40]
[43, 61, 55, 54, 58, 32, 37, 52, 43, 47]
[42, 59, 51, 51, 55, 27, 33, 51, 40, 40]
[43, 61, 55, 55, 58, 32, 37, 52, 43, 47]
[42, 59, 51, 51, 55, 27, 33, 51, 40, 41]
[43, 61, 55, 55, 58, 32, 37, 52, 43, 48]
[42, 59, 51, 51, 56, 27, 33, 51, 40, 41]
[43, 61, 55, 55, 59, 32, 37, 52, 43, 48]
[43, 59, 51, 51, 56, 27, 33, 51, 40, 41]
[44, 61, 55, 55, 59, 32, 37, 52, 43, 48]
[43, 59, 51, 52, 56, 27, 33, 51, 40, 41]
[44, 61, 55, 56, 59, 32, 37, 52, 43, 48]
[43, 59, 51, 52, 56, 27, 33, 52, 40, 41]
[44, 61, 55, 56, 59, 32, 37, 53, 43, 48]
[43, 59, 51, 53, 56, 27, 33, 52, 40, 41]
[44, 61, 55, 57, 59, 32, 37, 53, 43, 48]
[43, 59, 52, 53, 56, 27, 33, 52, 40, 41]
[44, 61, 56, 57, 59, 32, 37, 53, 43, 48]
[43, 59, 52, 53, 56, 27, 33, 53, 40, 41]
[44, 61, 56, 57, 59, 32, 37, 54, 43, 48]
[43, 59, 52, 53, 57, 27, 33, 53, 40, 41]
[44, 61, 56, 57, 60, 32, 37, 54, 43, 48]
[43, 59, 52, 54,

...

[98, 132, 117, 107, 107, 77, 74, 120, 96, 90]
[101, 135, 126, 112, 113, 86, 79, 123, 101, 105]
[98, 133, 117, 107, 107, 77, 74, 120, 96, 90]
[101, 136, 126, 112, 113, 86, 79, 123, 101, 105]
[98, 133, 117, 107, 107, 77, 74, 120, 96, 90]
[101, 136, 126, 112, 113, 86, 79, 123, 101, 106]
[98, 133, 117, 107, 108, 77, 74, 120, 96, 90]
[101, 136, 126, 112, 114, 86, 79, 123, 101, 106]
[98, 133, 117, 107, 108, 77, 74, 120, 96, 91]
[101, 136, 126, 112, 114, 86, 79, 123, 101, 107]
[98, 134, 117, 107, 108, 77, 74, 120, 96, 91]
[101, 137, 126, 112, 114, 86, 79, 123, 101, 107]
[98, 134, 117, 108, 108, 77, 74, 120, 96, 91]
[101, 137, 126, 113, 114, 86, 79, 123, 101, 107]
[98, 134, 117, 108, 108, 77, 74, 120, 96, 92]
[101, 137, 126, 113, 114, 86, 79, 123, 101, 108]
[98, 134, 118, 108, 108, 77, 74, 120, 96, 92]
[101, 137, 127, 113, 114, 86, 79, 123, 101, 108]
[99, 134, 118, 108, 108, 77, 74, 120, 96, 92]
[102, 137, 127, 113, 114, 86, 79, 123, 101, 108]
[99, 135, 118, 108, 108, 77, 74, 120, 96, 92]
[102

[113, 146, 124, 118, 122, 86, 82, 128, 107, 109]
[116, 149, 133, 123, 128, 95, 87, 132, 112, 126]
[113, 146, 124, 118, 122, 86, 83, 128, 107, 109]
[116, 149, 133, 123, 128, 95, 88, 132, 112, 126]
[113, 146, 124, 118, 122, 86, 83, 128, 107, 110]
[116, 149, 133, 123, 128, 95, 88, 132, 112, 127]
[113, 146, 125, 118, 122, 86, 83, 128, 107, 110]
[116, 149, 134, 123, 128, 95, 88, 132, 112, 127]
[113, 146, 125, 118, 122, 86, 84, 128, 107, 110]
[116, 149, 134, 123, 128, 95, 89, 132, 112, 127]
[113, 146, 126, 118, 122, 86, 84, 128, 107, 110]
[116, 149, 135, 123, 128, 95, 89, 132, 112, 127]
[113, 147, 126, 118, 122, 86, 84, 128, 107, 110]
[116, 150, 135, 123, 128, 95, 89, 132, 112, 127]
[113, 147, 127, 118, 122, 86, 84, 128, 107, 110]
[116, 150, 136, 123, 128, 95, 89, 132, 112, 127]
[114, 147, 127, 118, 122, 86, 84, 128, 107, 110]
[117, 150, 136, 123, 128, 95, 89, 132, 112, 127]
[114, 147, 127, 118, 122, 86, 84, 128, 108, 110]
[117, 150, 136, 123, 128, 95, 89, 132, 113, 127]
[114, 147, 127, 118,

[130, 159, 136, 127, 132, 95, 96, 140, 118, 121]
[133, 162, 146, 132, 138, 104, 101, 144, 123, 138]
[130, 160, 136, 127, 132, 95, 96, 140, 118, 121]
[133, 163, 146, 132, 138, 104, 101, 144, 123, 138]
[130, 160, 137, 127, 132, 95, 96, 140, 118, 121]
[133, 163, 147, 132, 138, 104, 101, 144, 123, 138]
[130, 160, 137, 128, 132, 95, 96, 140, 118, 121]
[133, 163, 147, 133, 138, 104, 101, 144, 123, 138]
[130, 160, 137, 128, 133, 95, 96, 140, 118, 121]
[133, 163, 147, 133, 139, 104, 101, 144, 123, 138]
[130, 160, 137, 128, 133, 96, 96, 140, 118, 121]
[133, 163, 147, 133, 139, 105, 101, 144, 123, 138]
[130, 160, 137, 128, 133, 96, 97, 140, 118, 121]
[133, 163, 147, 133, 139, 105, 102, 144, 123, 138]
[130, 160, 137, 128, 133, 96, 97, 141, 118, 121]
[133, 163, 147, 133, 139, 105, 102, 145, 123, 138]
[130, 160, 137, 128, 133, 96, 97, 141, 119, 121]
[133, 163, 147, 133, 139, 105, 102, 145, 124, 138]
[130, 160, 137, 128, 133, 96, 97, 141, 119, 122]
[133, 163, 147, 133, 139, 105, 102, 145, 124, 139]


[136, 169, 151, 138, 147, 107, 112, 147, 126, 138]
[139, 172, 161, 143, 154, 116, 117, 153, 131, 155]
[136, 169, 151, 139, 147, 107, 112, 147, 126, 138]
[139, 172, 161, 144, 154, 116, 117, 153, 131, 155]
[136, 169, 151, 139, 147, 107, 112, 147, 126, 139]
[139, 172, 161, 144, 154, 116, 117, 153, 131, 156]
[136, 169, 151, 140, 147, 107, 112, 147, 126, 139]
[139, 172, 161, 145, 154, 116, 117, 153, 131, 156]
[137, 169, 151, 140, 147, 107, 112, 147, 126, 139]
[140, 172, 161, 145, 154, 116, 117, 153, 131, 156]
[138, 169, 151, 140, 147, 107, 112, 147, 126, 139]
[141, 172, 161, 145, 154, 116, 117, 153, 131, 156]
[138, 170, 151, 140, 147, 107, 112, 147, 126, 139]
[141, 173, 161, 145, 154, 116, 117, 153, 131, 156]
[139, 170, 151, 140, 147, 107, 112, 147, 126, 139]
[142, 173, 161, 145, 154, 116, 117, 153, 131, 156]
[139, 170, 151, 140, 148, 107, 112, 147, 126, 139]
[142, 173, 161, 145, 155, 116, 117, 153, 131, 156]
[139, 170, 152, 140, 148, 107, 112, 147, 126, 139]
[142, 173, 162, 145, 155, 116, 

[151, 185, 172, 155, 176, 125, 127, 163, 142, 166]
[148, 182, 162, 150, 168, 115, 122, 157, 136, 148]
[151, 185, 172, 156, 176, 125, 127, 163, 142, 166]
[148, 182, 162, 150, 169, 115, 122, 157, 136, 148]
[151, 185, 172, 156, 177, 125, 127, 163, 142, 166]
[148, 182, 162, 150, 169, 116, 122, 157, 136, 148]
[151, 185, 172, 156, 177, 126, 127, 163, 142, 166]
[148, 182, 162, 150, 169, 116, 123, 157, 136, 148]
[151, 185, 172, 156, 177, 126, 128, 163, 142, 166]
[148, 182, 162, 150, 169, 116, 123, 158, 136, 148]
[151, 185, 172, 156, 177, 126, 128, 164, 142, 166]
[148, 182, 162, 150, 169, 116, 123, 158, 137, 148]
[151, 185, 172, 156, 177, 126, 128, 164, 143, 166]
[148, 182, 162, 150, 169, 116, 123, 158, 137, 149]
[151, 185, 172, 156, 177, 126, 128, 164, 143, 167]
[149, 182, 162, 150, 169, 116, 123, 158, 137, 149]
[152, 185, 172, 156, 177, 126, 128, 164, 143, 167]
Accuracy of 0: 0.980263
Accuracy of 1: 0.983784
Accuracy of 2: 0.941860
Accuracy of 3: 0.961538
Accuracy of 4: 0.954802
Accuracy of 5

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [12]:
from keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [13]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [ ]:
features_train, features_test, targets_train, targets_test = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42)

In [22]:
featuresTrain = torch.from_numpy(X_train)
targetsTrain = torch.from_numpy(Y_train).type(torch.LongTensor) # data type is long

featuresTest = torch.from_numpy(X_test)
targetsTest = torch.from_numpy(Y_test).type(torch.LongTensor) # data type is long

In [23]:
featuresTrain.shape, targetsTrain.shape, featuresTest.shape, targetsTest.shape

(torch.Size([60000, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 28, 28]),
 torch.Size([10000]))

In [24]:
# Pytorch train and test TensorDataset
train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

In [ ]:
# Hyper Parameters
# batch_size, epoch and iteration
LR = 0.01
batch_size = 100
n_iters = 10000
num_epochs = n_iters / (len(features_train) / batch_size)
num_epochs = int(num_epochs)

In [26]:
# Pytorch DataLoader
train_loader = torch.utils.data.DataLoader(train, batch_size = 32, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = 32, shuffle = True)

In [ ]:
# Create CNN Model
class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        # Convolution 1 , input_shape=(1,28,28)
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0) #output_shape=(16,24,24)
        self.relu1 = nn.ReLU() # activation
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2) #output_shape=(16,12,12)
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0) #output_shape=(32,8,8)
        self.relu2 = nn.ReLU() # activation
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2) #output_shape=(32,4,4)
        # Fully connected 1 ,#input_shape=(32*4*4)
        self.fc1 = nn.Linear(32 * 4 * 4, 10) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        # Max pool 1
        out = self.maxpool1(out)
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        # Max pool 2 
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        # Linear function (readout)
        out = self.fc1(out)
        return out

In [ ]:
model = CNN_Model()
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)   # optimize all cnn parameters
loss_func = nn.CrossEntropyLoss()   # the target label is not one-hotted
input_shape = (-1,1,28,28)

In [ ]:
def fit_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, test_loader):
    # Traning the Model
    #history-like list for store loss & acc value
    training_loss = []
    training_accuracy = []
    validation_loss = []
    validation_accuracy = []
    for epoch in range(num_epochs):
        #training model & store loss & acc / epoch
        correct_train = 0
        total_train = 0
        for i, (images, labels) in enumerate(train_loader):
            # 1.Define variables
            train = Variable(images.view(input_shape))
            labels = Variable(labels)
            # 2.Clear gradients
            optimizer.zero_grad()
            # 3.Forward propagation
            outputs = model(train)
            # 4.Calculate softmax and cross entropy loss
            train_loss = loss_func(outputs, labels)
            # 5.Calculate gradients
            train_loss.backward()
            # 6.Update parameters
            optimizer.step()
            # 7.Get predictions from the maximum value
            predicted = torch.max(outputs.data, 1)[1]
            # 8.Total number of labels
            total_train += len(labels)
            # 9.Total correct predictions
            correct_train += (predicted == labels).float().sum()
        #10.store val_acc / epoch
        train_accuracy = 100 * correct_train / float(total_train)
        training_accuracy.append(train_accuracy)
        # 11.store loss / epoch
        training_loss.append(train_loss.data)

        #evaluate model & store loss & acc / epoch
        correct_test = 0
        total_test = 0
        for images, labels in test_loader:
            # 1.Define variables
            test = Variable(images.view(input_shape))
            # 2.Forward propagation
            outputs = model(test)
            # 3.Calculate softmax and cross entropy loss
            val_loss = loss_func(outputs, labels)
            # 4.Get predictions from the maximum value
            predicted = torch.max(outputs.data, 1)[1]
            # 5.Total number of labels
            total_test += len(labels)
            # 6.Total correct predictions
            correct_test += (predicted == labels).float().sum()
        #6.store val_acc / epoch
        val_accuracy = 100 * correct_test / float(total_test)
        validation_accuracy.append(val_accuracy)
        # 11.store val_loss / epoch
        validation_loss.append(val_loss.data)
        print('Train Epoch: {}/{} Traing_Loss: {} Traing_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
    return training_loss, training_accuracy, validation_loss, validation_accuracy

In [ ]:
training_loss, training_accuracy, validation_loss, validation_accuracy = fit_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, test_loader)

In [ ]:
# visualization
plt.plot(range(num_epochs), training_loss, 'b-', label='Training_loss')
plt.plot(range(num_epochs), validation_loss, 'g-', label='validation_loss')
plt.title('Training & Validation loss')
plt.xlabel('Number of epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.plot(range(num_epochs), training_accuracy, 'b-', label='Training_accuracy')
plt.plot(range(num_epochs), validation_accuracy, 'g-', label='Validation_accuracy')
plt.title('Training & Validation accuracy')
plt.xlabel('Number of epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()